<a href="https://colab.research.google.com/github/Saylion/Drive-to-Huggingface-Uploader-Colab/blob/main/Drive_to_Huggingface_Uploader_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Input Form

Huggingface_token = "" #@param {type:"string"}
Is_this_folder_or_file_transfer = "folder" #@param ["file", "folder"]
Your_source_path = "" #@param {type:"string"}
#@markdown fill this section with your destination path on Huggingface
Your_destination_path = "" #@param {type:"string"}
Your_repo_type = "model" #@param ["chose your repo model","model","dataset","space"]
RVC_sort = False #@param {type:"boolean"}
#@markdown (if RVC sort is True, the input of source path will be disabled)

y_token = Huggingface_token
src_path = Your_source_path
dest_path = Your_destination_path
rep_type = Your_repo_type
transfer_type = Is_this_folder_or_file_transfer

path = dest_path.split("/", maxsplit=2)

splitted = src_path.split('/')
filename = splitted[-1]
if len(path) == 2:
  rep_path = filename
elif len(path) > 2:
  rep_path = path[2] + "/" + filename
rep_id = path[0] + "/" + path[1]
key_log = 0 #this variable is for detect if RVC section has running

In [ ]:
#@title Install HuggingFace Hub
!pip install -U "huggingface_hub[cli]"

In [ ]:
#@title Auto Login
from huggingface_hub import login
from huggingface_hub import HfApi
api = HfApi()

login(token=y_token)

from huggingface_hub import whoami

user_info = whoami()
print(f"\nLogged in as: {user_info['name']} ({user_info['fullname']})")

In [ ]:
#@title This Section is for sort valid RVC Model (Optional)
# @markdown This section will also collect the .index and .pth files from defaut RVC dirrectory in Gdrive (/content/drive/MyDrive/RVC)in one model folder (You can skip this section)

#sort valid RVC Model-----
if RVC_sort == True:
  import numpy as np
  import os
  from IPython.display import clear_output

  Default_RVC_path = "/content/drive/MyDrive/RVC"

  folders = os.listdir(Default_RVC_path)
  array_folders = []
  for folder in folders:
    if not os.path.isfile(os.path.join(Default_RVC_path, folder)):
      array_folders.append(folder)

  list_folder = np.array(array_folders)

  for folder in list_folder:
    if not folder == "weights" and not folder == ".ipynb_checkpoints":
      !mkdir -p /content/Temporary/rvc_models/{folder}/
      !cp /content/drive/MyDrive/RVC/{folder}/added_*.index /content/Temporary/rvc_models/{folder}/
      !cp -r /content/drive/MyDrive/RVC/weights/{folder}.pth /content/Temporary/rvc_models/{folder}


  %cd /content/Temporary/rvc_models
  !find . -type d -exec bash -c '[[ $(find "$1" -maxdepth 1 -type f | wc -l) -lt 2 ]]' _ {} \; -print0 | xargs -0 rm -rf

  clear_output()
  print("RVC Sort Process DONE\n\n")

  print("Starting upload to Huggingface...")

  temporary_path = "/content/Temporary/rvc_models"
  folders = os.listdir(temporary_path)
  array_folders = []
  for folder in folders:
    if not os.path.isfile(os.path.join(temporary_path, folder)):
      array_folders.append(folder)

  list_folder = np.array(array_folders)

  for folder in list_folder:

    if not folder == ".ipynb_checkpoints":
      final_path = temporary_path + "/" + folder
      final_rep_path = "Models" + "/" + folder
      api.upload_folder(
        folder_path=final_path,
        path_in_repo=final_rep_path,
        repo_id=rep_id,
        repo_type=rep_type,
      )

  print("All folder and file has been uploaded")
  key_log = 1

In [ ]:
#@title Reguler Uploader

if key_log == 1:
  print("You have upload folder by RVC sort section, run again input Section and back to this section")

else:
  print("Starting upload to Huggingface...")
  if transfer_type == "file":
    api.upload_file(
        path_or_fileobj=src_path,
        path_in_repo=rep_path,
        repo_id=rep_id,
        repo_type=rep_type,
    )
    print("File has been uploaded")

  else:
    api.upload_folder(
        folder_path=src_path,
        path_in_repo=rep_path,
        repo_id=rep_id,
        repo_type=rep_type,
    )
    print("All folder and file has been uploaded")